## Read PRIM master file to df

In [ ]:
#@time begin
using DataFrames, Dates
using NativeFileDialog
using Plots, Printf
using Statistics

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# select PRIM master file to be read
infil = pick_file("C:\\QGHL\\Wave_data\\Master_files\\", filterlist="prim");
println("Selected ",infil)
flush(stdout)

# Create a df to hold the parameters read from the binary master file
col_names = [:Date, :Loc, :Hsig, :Thsig, :Hrms, :Hmax, :Tc, :Tz, :H10, :TH10, :Thmax, 
    :Tzmax, :Hm0, :T02, :Tp, :Eps, :Npts]
col_types = [Date, Int, Float32, Float32, Float32, Float32, Float32, Float32, 
    Float32, Float32, Float32, Float32, Float32, Float32, Float32, Float32, Int]
named_tuple = (; zip(col_names, type[] for type in col_types )...)

df = DataFrame(named_tuple)
@time begin
# read the contents of the PRIM master file
open(infil) do io
    
    i = 0
    while !eof(io)
#==        
        if mod(i,30)==0 
            
            println("         Date        Loc  Hsig  THsig   Hrms   Hmax    Tc     Tz     H10   TH10  THmax  Tzmax    Hm0    T02    Tp     Eps Points")

        end
==#    
        ProcId = read(io, 12)
        Loc = read(io, Int32)
        year = read(io, Int32)
        mth =  read(io, Int32)
        day =  read(io, Int32)
        hr =  read(io, Int32)
        min =  read(io, Int32)
        sec =  read(io, Int32)
        hsig = read(io, Float32)
        thsig = read(io, Float32)
        hrms = read(io, Float32)
        hmax = read(io, Float32)
        tc = read(io, Float32)
        tz = read(io, Float32)
        h10 = read(io, Float32)
        th10 = read(io, Float32)
        thmax = read(io, Float32)
        tzmax = read(io, Float32)
        hm0 = read(io, Float32)
        t02 = read(io, Float32)
        tp = read(io, Float32)
        eps = read(io, Float32)
        npts = read(io, Int32)
        param2 = read(io, 16)
        
        # calculate record date
        date = DateTime(year, mth, day, hr, min, sec)

        push!(df, [date,Loc,hsig,thsig,hrms,hmax,tc,tz,h10,th10,thmax,tzmax,hm0,t02,tp,eps,npts])
##        @printf("%s %3d %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f %5d\n",
##            date,Loc,hsig,thsig,hrms,hmax,tc,tz,h10,th10,thmax,tzmax,hm0,t02,tp,eps,npts)
            
        i+=1
        
    end

end
end
##close(io)

# PLot Hs, Hmax, Tz, Tp
p1 = plot(df.Date,df.Hmax, label="Hmax", title=splitdir(infil)[end])
p1 = plot!(df.Date,df.Hsig, label="Hsig")

p2 = plot(df.Date,df.Tp, label="Tp")
p2 = plot!(df.Date,df.Tz, label="Tz")

p1_p2_plot = plot(p1, p2, layout=(2,1), size=(1800,800), xlims=(first(df.Date),last(df.Date)),
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, bottommargin=0*Plots.mm,)
    
display(p1_p2_plot)


In [ ]:
# Read data from RTWS binary PRIM Master file
using DataFrames, Dates
using NativeFileDialog
using Plots, Printf
using Statistics
using StructIO, StaticStrings, StaticArrays


function get_dates(date_vector)
########################################    
# convert SVector to DateTime

    DateTime(date_vector[1],date_vector[2],date_vector[3],date_vector[4],date_vector[5],date_vector[6])

end

# Create a structure to hold parameters read from PRIM Master file    
@io struct Frame
    ProcId::StaticString{12}
    Loc::SVector{1, Int32}
    Date::SVector{6, Int32}
    Value::SVector{14, Float32}
    npts::SVector{1, Int32}
    param2::StaticString{16}
end align_packed
    

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# select PRIM Master file to be read
infil = pick_file("C:\\QGHL\\Wave_data\\Master_files\\", filterlist="prim");
println("Selected ",infil)
flush(stdout)

data = reinterpret(Frame, read(infil));

# Create empty df
df = DataFrame() 

# Names of df columns
columns = ["Date","Loc","Hsig","THsig","Hrms","Hmax","Tc","Tz","H10","TH10","THmax","Tzmax","Hm0",
        "T02","Tp","Eps","Points"]

# Populate the df with data from the Master file
df[!, columns[1]] = get_dates.((Frame->Frame.Date).(data));
df[!, columns[2]] = (Frame->Frame.Loc[1]).(data);

[df[!, columns[i]] = (Frame->Frame.Value[i-2]).(data) for i in 3:16];
df[!, columns[17]] = (Frame->Frame.npts[1]).(data);

# PLot Hs, Hmax, Tz, Tp
p1 = plot(df.Date,df.Hmax, label="Hmax", title=splitdir(infil)[end])
p1 = plot!(df.Date,df.Hsig, label="Hsig")

p2 = plot(df.Date,df.Tp, label="Tp")
p2 = plot!(df.Date,df.Tz, label="Tz")

p1_p2_plot = plot(p1, p2, layout=(2,1), size=(1800,800), xlims=(first(df.Date),last(df.Date)),
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, bottommargin=0*Plots.mm,)
    
display(p1_p2_plot)

In [ ]:
function ratios(A, B)
    
    ratio = A ./ B
    med = median(ratio)
    stdev = std(ratio)
    
    return(ratio, med, 3 .* stdev)
    
end

# Ratio Hmax/Hsig
ratio,med,stdev3  = ratios(df.Hmax, df.Hsig)
p1a = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio Hmax / Hsig")
p1a = hline!([med], lc=:blue, label="Median")
p1a = hline!([med .+ stdev3], lc=:red, label="3σ")
p1a = hline!([med .- stdev3], lc=:red, label="")

# Ratio Hrms/Hsig
ratio,med,stdev3  = ratios(df.Hrms, df.Hsig)
p2a = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio Hrms / Hsig")
p2a = hline!([med], lc=:blue, label="Median")
p2a = hline!([med .+ stdev3], lc=:red, label="3σ")
p2a = hline!([med .- stdev3], lc=:red, label="")

# Ratio Hm0/Hsig
ratio,med,stdev3  = ratios(df.Hm0, df.Hsig)
p3a = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio Hm0 / Hsig")
p3a = hline!([med], lc=:blue, label="Median")
p3a = hline!([med .+ stdev3], lc=:red, label="3σ")
p3a = hline!([med .- stdev3], lc=:red, label="")

# Ratio H10/Hsig
ratio,med,stdev3  = ratios(df.H10, df.Hsig)
p4a = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio H10 / Hsig")
p4a = hline!([med], lc=:blue, label="Median")
p4a = hline!([med .+ stdev3], lc=:red, label="3σ")
p4a = hline!([med .- stdev3], lc=:red, label="")

# Ratio Tp/Tz
ratio,med,stdev3  = ratios(df.Tp, df.Tz)
p1b = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio Tp / Tz")
p1b = hline!([med], lc=:blue, label="Median")
p1b = hline!([med .+ stdev3], lc=:red, label="3σ")
p1b = hline!([med .- stdev3], lc=:red, label="")

# Ratio T02/Tz
ratio,med,stdev3  = ratios(df.T02, df.Tz)
p2b = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio T02 / Tz")
p2b = hline!([med], lc=:blue, label="Median")
p2b = hline!([med .+ stdev3], lc=:red, label="3σ")
p2b = hline!([med .- stdev3], lc=:red, label="")

# Ratio Tzmax/Tz
ratio,med,stdev3  = ratios(df.Tzmax, df.Tz)
p3b = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio Tzmax / Tz")
p3b = hline!([med], lc=:blue, label="Median")
p3b = hline!([med .+ stdev3], lc=:red, label="3σ")
p3b = hline!([med .- stdev3], lc=:red, label="")

# Ratio Tc/Tz
ratio,med,stdev3  = ratios(df.Tc, df.Tz)
p4b = scatter(df.Date,ratio, ms=:2, msw=:0, label="", title="Ratio Tc / Tz")
p4b = hline!([med], lc=:blue, label="Median")
p4b = hline!([med .+ stdev3], lc=:red, label="3σ")
p4b = hline!([med .- stdev3], lc=:red, label="")

plot_ratios = plot(p1a, p2a, p3a, p4a, p1b, p2b, p3b, p4b,
    layout=(2,4), size=(1800,800), titlefontsize=10, 
    xlims=(first(df.Date),last(df.Date)),
    framestyle = :box,fg_legend=:transparent, bg_legend=:transparent, bottommargin=10*Plots.mm,)

display(plot_ratios)

## Read SECO master file

In [ ]:
using DataFrames, Dates
using NativeFileDialog
using Plots, Printf
using Statistics

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# select SECO master file to be read
infil = pick_file("C:\\QGHL\\Wave_data\\Master_files\\", filterlist="seco");
println("Selected ",infil)
flush(stdout)

# Create a df to hold the parameters read from the binary master file
col_names = [:Date, :Loc, :Spc]
col_types = [Date, Int, Float32]
named_tuple = (; zip(col_names, type[] for type in col_types )...)

#df = DataFrame(named_tuple)
df = DataFrame([[],[],[]], ["Date", "Loc", "Spc"])
# read the contents of the SECO master file
open(infil) do io
    
    while !eof(io)
        
        Loc = read(io, Int32)
        year = read(io, Int32)
        mth =  read(io, Int32)
        day =  read(io, Int32)
        hr =  read(io, Int32)
        min =  read(io, Int32)
        sec =  read(io, Int32)
        
        Spc = Vector{Float32}()
        for i in 1:60
            aa = read(io, Float32)
            append!(Spc,aa)
        end     
        
        # calculate record date
        date = DateTime(year, mth, day, hr, min, sec)

        push!(df, [date,Loc,Spc])
        
    end

end


Selected C:\QGHL\Wave_data\Master_files\mool----04.seco


In [ ]:
function monthly_contour_plot(ii, monthly, freq)
############################################    
# do a contour plot for a month
    
    date_string = Dates.monthname(monthly[ii].Date[1])*" "*string(Dates.year(monthly[ii].Date[1]))

    # convert the spectral df data into a matrix
    ll = nrow(monthly[ii])
    
    spec_matrix = monthly[ii].Spc[1]'

    for jj in 2:ll
        spec_matrix = vcat(spec_matrix, monthly[ii].Spc[jj]');
    end

    x = 1:1:ll #sort(monthly.Date[2:ll], rev=:false)
    y = freq
    z = spec_matrix'

##    global tm_tick = range(1, ll, step=48) #range(monthly.Date[2], monthly.Date[ll], step=Day(2))
##    global ticks = unique(Dates.day.(monthly[ii].Date[1:ll])) #Dates.format.(tm_tick,"dd")
    
    ticks = unique(Dates.day.(monthly[ii].Date[1:ll])) #Dates.format.(tm_tick,"dd")
    tm_tick = range(1, ll, step=ll/length(ticks)) #range(monthly.Date[2], monthly.Date[ll], step=Day(2))

    # do a contour plot of the matrix data - i.e. spectral data pploted over one day

    c1 = contourf(x, y, z, lw=:0.5, nlevels=100, colormap=cgrad(:Spectral_11, rev=:true), clabels=:false, alpha=0.95, 
        xticks=(tm_tick,ticks), xlim=(first(x), last(x)), xminorgrid=:true, xminorticks=:4, 
        ylim=(0,0.4), 
        cbar=:false, cbar_title="Spectral Density (m²/Hz.)", colorbar_titlefontsize=10, colorbar_titlefontrotation=-180, 
#        leftmargin=10*Plots.mm, rightmargin=10*Plots.mm, bottommargin=7.5*Plots.mm, topmargin=0*Plots.mm,
        fg_legend=:transparent, bg_legend=:transparent, title=date_string, titlefontsize=:10, framestyle = :box)

    c1 = contour!(x, y, z, lc=:lightblue, lw=:0.5)

    for i in sort(0.1:0.1:1.0)
        c1 = hline!([i], lw=:0.5, lc=:lightgrey, label="")
    end

    for i in sort(tm_tick)
        c1 = vline!([i], lw=:0.5, lc=:lightgrey, label="")
    end
    
    return(c1)

end    # monthly_contour_plot()

freq = []
for i in 1:60 append!(freq, 0.025+i*0.01) end

for kk in 2015 #unique(year.(df.Date))
        
    # get df data for year of interest
    yoi = kk #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
    yoi_string = string(yoi)
##    println("Selected ",site," for year ",yoi_string)
    year_df = df[year.(df.Date) .== yoi,:]

    # group rows in spectra_df into daily sets of records
    monthly_df = groupby(transform(year_df, :Date => x->yearmonth.(x)),:Date_function);
    months = unique(month.(year_df.Date));

    println("Preparing plots now ... takes a while!")
    flush(stdout)

    # create an array to hold the monthly subplots
    plot_arry = []
    for ii in 1:length(months)

        push!(plot_arry,monthly_contour_plot(ii, monthly_df, freq))

    end

    # plot the monthly results from the array using the splat command (...)
    plot_all = plot(plot_arry..., size=(1800, 250*length(months)), layout=(length(months),1),
        ylabel="Frequency (Hz.)",
        leftmargin=5*Plots.mm, bottommargin=0*Plots.mm, topmargin=0*Plots.mm,)

##    savefig(".\\Spectra_"*site*"_"*yoi_string*"_contour.png")

    display(plot_all)

end

In [ ]:
infil

## Read DSECO master file

In [ ]:
using DataFrames, Dates
using NativeFileDialog
using Plots, Printf
using Statistics

# Widen screen for better viewing
display("text/html", "<style>.container { width:100% !important; }</style>")

# select DSECO master file to be read
#infil = pick_file("C:\\QGHL\\Wave_data\\Master_files\\", filterlist="dseco");
infil = "C:\\QGHL\\Wave_data\\RTWS\\TWEE\\twee----28.dseco"
println("Selected ",infil)
flush(stdout)

dseco_df = DataFrame([[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]], 
    ["Date", "Loc", "Elapse", "Hm0", "T02", "Pdens", "Temp", "Vacc", "Xacc", "Yacc", "Orient", "Inclin", "Tp_buoy", "Avg_dir", "Peak_dir", "Corr_to_True", "Jdirn", "Jrpower", "Jspread"])

# read the contents of the DSECO master file
open(infil) do io
    
    while !eof(io)
        
        ProcId = read(io, 12)
        Loc = read(io, Int32)
        year = read(io, Int32)
        if year < 1900
            year += 1900
        end
        mth =  read(io, Int32)
        day =  read(io, Int32)
        hr =  read(io, Int32)
        min =  read(io, Int32)
        sec =  read(io, Int32)
        elapse = read(io, Float32)
        hm0 = read(io, Float32)
        t02 = read(io, Float32)
        Pdens = read(io, Float32)
        temp = read(io, Float32)
        vacc = read(io, Float32)
        xacc = read(io, Float32)
        yacc = read(io, Float32)
        orient = read(io, Float32)
        inclin = read(io, Float32)
        tp_buoy = read(io, Float32)
        avg_dir = read(io, Float32)
        peak_dir = read(io, Float32)
        corr_to_true = read(io, Float32)
        
        jdirn = Vector{Int16}()
        [append!(jdirn,read(io, Int16)) for i in 1:62]
        [append!(jdirn,0) for i in 1:2]
        
        jrpower = Vector{Int16}()
        [append!(jrpower,read(io, Int16)) for i in 1:64]
        
        jspread = Vector{Int16}()
        [append!(jspread,read(io, Int16)) for i in 1:64]
                    
        # calculate record date
        date = DateTime(year, mth, day, hr, min, sec)

        push!(dseco_df, [date,Loc,elapse,hm0,t02,Pdens,temp,vacc,xacc,yacc,orient,inclin,tp_buoy,avg_dir,peak_dir,corr_to_true,jdirn,jrpower,jspread])
        
    end

end


In [ ]:
#==
Apply translation equations for Dseco parameters 
Refer Datawell Waverider Reference Manual MkIII February 1, 2020 p.50 Table 5.7.3
==#
rec = 11936

power = exp.(-dseco_df.Jrpower[rec]./200) .* dseco_df.Pdens[rec]
dirn = dseco_df.Jdirn[rec] .* 360 ./ 256

if (abs(dseco_df.Corr_to_True[rec]) < 0.0001) 
    dir_val = "True"
else
    dir_val = "Magnetic"
end

spread = dseco_df.Jspread[rec] .* 0.4476

freq = []
        
for i in 1:64
    
    if i<16
        append!(freq, 0.025+i*0.005) 
    else
        append!(freq, 0.11+(i-16)*0.01)
    end

end
    
spread_hi = dirn+spread
spread_lo = dirn-spread


date_string = Dates.format(dseco_df.Date[rec], "yyyy-mm-dd HH:MM:SS")
title=splitdir(infil)[end] * " " * date_string
p1 = plot(freq, dirn, lw=:2, lc=:blue, label="Dirn", ylim=(0,360), legend=:topleft, xaxis="Frequency (Hz.)", yaxis="Direction (ᵒ "*dir_val*")")

p1 = plot!(freq, spread_hi, lc=:blue, alpha=:0.5, ls=:dot, label="")    
p1 = plot!(freq, spread_lo, lc=:blue, alpha=:0.5, ls=:dot, label="") 

# get plotting limits, and display input file name at top left of plot
x_lim1 = xlims(p1)[1]; y_lim1 = ylims(p1)[1]
x_lim2 = xlims(p1)[2]; y_lim2 = ylims(p1)[2]   
x_pos = x_lim1 + abs(x_lim2-x_lim1)*0.75; y_pos = y_lim2*0.75
file_name_string = replace(infil, "\\" => "/")

# Display some parameters on plot
p1 = annotate!(x_pos, y_pos, text("Hm0 = "*string(round(dseco_df.Hm0[rec], digits=1))
    *"\n\nT02 = "*string(round(dseco_df.T02[rec], digits=1))
    *"\n\nTp = "*string(round(dseco_df.Tp_buoy[rec], digits=1))
    *"\n\nAvg. Dirn. = "*string(round(dseco_df.Avg_dir[rec], digits=1))
    *"\n\nPeak Dirn. = "*string(round(dseco_df.Peak_dir[rec], digits=1)), :red, :left, 8))


p1 = plot!(twinx(), freq, power, lw=:2, lc=:red, label="Power", ylim=(0,dseco_df.Pdens[rec]*1.05), 
    legend=:topright, yaxis="Spectral Density (m²/Hz.)", yguidefontrotation=-180)

p1 = vline!([1/dseco_df.Tp_buoy[rec]], label="")
plot(p1, framestyle = :box, leftmargin=7.5*Plots.mm, rightmargin=10*Plots.mm, bottommargin=7.5*Plots.mm, topmargin=0*Plots.mm,
    fg_legend=:transparent, bg_legend=:transparent, title=title, titlefontsize=:10, size=(1400,600))

## Contour plot of daily spectra

In [ ]:
using Tk

# Convert datetime array to string array
#date_df = dseco_df.Date
dates = Dates.format.(unique([Date(d) for d in dseco_df.Date]), "yyyy-mm-dd");

##max_pdens = maximum(dseco_df.Pdens)
##min_pdens = minimum(dseco_df.Pdens)

w = Toplevel("Select Date", 235, 600)
tcl("pack", "propagate", w, false)
ff = Frame(w)
pack(ff, expand=true, fill="both")

f1 = Frame(ff)
lb = Treeview(f1, dates)
scrollbars_add(f1, lb)
pack(f1,  expand=true, fill="both")

tcl("ttk::style", "configure", "TButton", foreground="blue", font="arial 16 bold")
b = Button(ff, "Ok")
pack(b)

bind(b, "command") do path
    
    date_choice = get_value(lb)
    global idx = indexin(date_choice, dates)
    idx = idx[1]
    
    global start_date = Date.(dates[idx], "yyyy-mm-dd")
    global end_date = start_date + Day(1)
    
    global day_df = dseco_df[@.(start_date <= dseco_df.Date < end_date), :]
    
    ll = nrow(day_df)
    aa = (exp.(-day_df.Jrpower[1]./200) .* day_df.Pdens[1])'

    for i in 2:ll
        aa = vcat(aa, (exp.(-day_df.Jrpower[i]./200) .* day_df.Pdens[i])');
    end
    
    freq = []
        
    for i in 1:64

        if i<16
            append!(freq, 0.025+i*0.005) 
        else
            append!(freq, 0.11+(i-16)*0.01)
        end

    end

    x = day_df.Date[1:ll]
    y = freq
    z = aa'

    # display plots to screen
    global tm_tick = range(first(x), last(x), step=Hour(4))
    global ticks = Dates.format.(tm_tick,"dd-mm-yy HH:MM")

    # do the contour plot of selected data
    p1 = contourf(x, y, z, zcolor = z, lw=:0, nlevels=:50, colormap=cgrad(:RdYlBu_11, rev=:true), clabels=:false, alpha=0.95, 
    cbar_title="Spectral Density (m²/Hz.)", colorbar_titlefontsize=10, colorbar_titlefontrotation=-180)
    p1 = contour!(x, y, z, lc=:lightblue, lw=:0.5)

    for i in tm_tick
        p1 = vline!([i], label="", lc=:white, lw=:0.5, ls=:dot)
    end

    for i in 0:0.1:0.64
        p1 = hline!([i], label="", lc=:white, lw=:0.5, ls=:dot)
    end


    do_p1 = plot(p1, size=(1800,800), ylabel="Frequency (Hz.)", framestyle = :box, leftmargin=7.5*Plots.mm, rightmargin=7.5*Plots.mm, bottommargin=7.5*Plots.mm, topmargin=0*Plots.mm,
        xticks=(tm_tick,ticks), xlim=(first(x), last(x)), xminorgrid=:true, xminorticks=:4, ylim=(0,0.4),
        fg_legend=:transparent, bg_legend=:transparent, title=dates[idx], titlefontsize=:10)
    
    display(do_p1)              
    
end